Import libraries and use Beautiful Soup to get table from web page

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
 
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')

In [2]:
table = soup.find('table', {'class':'wikitable sortable'})

In [3]:
links = table.findAll('a')

Append values from the table into three arrays and create data frame with these arrays

In [4]:
Postal_codes = []
Boroughs = []
Neighbourhoodsraw = []
Neighbourhoods = []

for row in table.findAll('tr'):
    cells = row.findAll('td')
    if len(cells)==3:
        Postal_codes.append(cells[0].find(text=True))
        Boroughs.append(cells[1].find(text=True))
        Neighbourhoodsraw.append(cells[2].find(text=True))

df = pd.DataFrame()
df['Postal Codes'] = Postal_codes
df['Borough'] = Boroughs
df['Neighbourhood'] = Neighbourhoodsraw

Remove rows where Borough is not assigned

In [5]:
df = df[~df.Borough.str.contains('Not assigned')]

For values in not assigned in Neighbourhood column, assign the same Borough value.
Then remove \n from strings 

In [6]:
for i, j in df.iterrows():
    if j['Neighbourhood'] == 'Not assigned\n':
        j['Neighbourhood'] = j['Borough']
        
for i in df['Neighbourhood']:
    if '\n' in i:
        i = i[:-1]
        Neighbourhoods.append(i)
    else:
        Neighbourhoods.append(i)
        

In [7]:
df['Neighbourhood'] = Neighbourhoods

Group all neighbourhoods with same postal code

In [8]:
df = df.groupby(['Postal Codes', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()


In [9]:
df

,Postal Codes,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


In [10]:
df.shape

(103, 3)

Use Geocoder to get longitude/latitude
Since both dataframes are grouped by Postal Code, we can copy the two wanted columns

In [11]:
locations = pd.read_csv('Geospatial_Coordinates.csv') 

In [12]:
locations

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [13]:
df['Latitude'] = locations['Latitude']
df['Longitude'] = locations['Longitude']

In [15]:
df

,Postal Codes,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437


# Mapping the Neighbourhoods

Import tools neccessary to produce map and clusters

In [20]:
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
from geopy.geocoders import Nominatim

Create a map, marking the different postal coded areas

In [24]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitudes = location.latitude
longitudes = location.longitude

In [28]:
tor_map = folium.Map(location =[latitudes, longitudes], zoom_start = 10)

for lat, lng, borough, neighbourhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(tor_map)  
tor_map

Splitting up the clusters by their boroughs, I chose Scarborough as an example. 

Given any choice, we should expect to see that the markers would be clustered around a certain area with no outliers across the other side of the city. 

In [34]:
Scarborough_data = df[df['Borough'] == 'Scarborough'].reset_index(drop=True)
etobicoke_data = df[df['Borough'] == 'Etobicoke'].reset_index(drop=True)

In [31]:
address = 'Scarborough, ON'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [33]:
scarborough_map = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, label in zip(Scarborough_data['Latitude'], Scarborough_data['Longitude'], Scarborough_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(scarborough_map)  
scarborough_map

As one would expect, the Scarborough area is clustered around eastern toronto. 

Taking another example in Etobicoke, using a similar method. 

In [35]:
etobicoke_map = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, label in zip(etobicoke_data['Latitude'], etobicoke_data['Longitude'], etobicoke_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(etobicoke_map)  
etobicoke_map

Again we see what we should expect, with a cluster on the western end of Toronto